In [1]:
!pip install -e C:\Users\asus_strix_scar\Anaconda3\Lib\site-packages\gym\envs\steering-optimizer\steering_optimizer\envs\github\steering-optimizer\

Obtaining file:///C:/Users/asus_strix_scar/Anaconda3/Lib/site-packages/gym/envs/steering-optimizer/steering_optimizer/envs/github/steering-optimizer
  Found existing installation: steering-optimizer 0.0.1
    Uninstalling steering-optimizer-0.0.1:
      Successfully uninstalled steering-optimizer-0.0.1
  Running setup.py develop for steering-optimizer


In [2]:
import numpy as np

In [3]:
import gym

In [4]:
for env in gym.envs.registry.env_specs.keys():
     if 'steering_optimizer' in env:
          print('Remove {} from registry".format(env)')
          del gym.envs.registry.env_specs[env]

In [5]:
from gym.envs.registration import register

register(
    id='steering_optimizer-v0',
    entry_point='steering_optimizer.envs:StrOptEnv',# kwargs={'WB': 1900, 'TW': 1200, 'KP': 150,'tr_min': 4000}
)

In [6]:
env = gym.make("steering_optimizer-v0")

In [7]:
#Fixed parameters

#Wheelbase, track width and kingpin distance

WB = 1900
TW = 1200
KP = 150

In [8]:
batch_size = 100
session_size = 100
t_max = 300
percentile = 90
hidden_size = 20
learning_rate = 0.01
completion_score = 200

In [9]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class Net(nn.Module):
        
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.fc0 = nn.Linear(obs_size, hidden_size)
        self.fc1 = nn.Linear(hidden_size, n_actions)
        
    def forward(self, x):
        activation = nn.Softmax(dim=1)
        
        x = F.relu(self.fc0(x))
        
        return activation(self.fc1(x))

    def generate_batch(self, env, batch_size, t_max):
        
        #activation_1 = nn.Softmax(dim=1)
        batch_actions, batch_states, batch_rewards = [],[],[]
        # Reset only once in a batch
        s_0 = env.reset()
        
        for b in range(batch_size):
            states,actions = [],[]
            total_reward = 0
            
            s = env.reset()
            # Imitating reset, with the same state for every batch member
            #env.state = s_0
            
            #s = s_0
            
            #env.steps_beyond_done = None
            #env.error = None
            #env.max_r = None
            #env.steps_since_reset = 0
            #env.total_reward = 0
            
            #print('state shape after reset:', s.shape)
            s = s.reshape((1,4))
            
            #print('state shape after reshape:', s.shape)
            
            # step counter
            count = 0
            
            for t in range(t_max):
                count +=1
                
                # Scaling state to be within -1 and 1
                s = s/TW*2
                
                s_v = torch.FloatTensor([s]).reshape(((1,4)))
                #print('state_shape',s_v.shape)
                #s_v = torch.FloatTensor([s])
                
                #print('state: x',s_v)
                #act_probs_v = activation_1(self.forward(s_v))
                act_probs_v = self.forward(s_v)
                act_probs = act_probs_v.data.numpy()[0]
                choice_num = len(act_probs)
                #print('num:',choice_num)
                
                ## Making only 
                #env.steps_since_reset += 1
                
                #r = -1
                
                #while r <= 0:
                #    env.steps_since_reset -= 1
                #    a = np.random.choice(choice_num, p=act_probs)
                #    
                #    new_s, r, done, info = env.step(a)
                # 
                
                a = np.random.choice(choice_num, p=act_probs)
                #a = np.random.choice(choice_num)
                   
                new_s, r, done, info = env.step(a)

                states.append(s)
                actions.append(a)
                
                #print('new state',new_s)
                
                total_reward += r
                
                #print('action:',a, 'reward: ', r, 'error:', env.error, 'total reward:', total_reward)
                
                # print('total reward:', total_reward)
                # Getting the new state               
                s = new_s/TW*2
                
                # After done, no more steps
                if done:
                    #print('DONE happened')
                    break
            
            batch_actions.append(actions)
            #print('Batch member done')
            #print('Batch actions:', batch_actions[-1])
            #print('total reward on episode:', total_reward)
            
            batch_states.append(states)
            batch_rewards.append(total_reward)
        
        #print('Batch generated')
        #print('States:', batch_states)
        #print('Episode Actions:', batch_actions)
        #print('Rewards:', batch_rewards)
        
        return batch_states, batch_actions, batch_rewards

    def filter_batch(self, states_batch, actions_batch, rewards_batch, percentile):
        
        reward_threshold = np.percentile(rewards_batch, percentile)
        
        #print('reward threshold:', reward_threshold)
        
        threshold_cnt = 0
        for i in range(len(rewards_batch)):
            if rewards_batch[i] > reward_threshold:
                threshold_cnt += 1
                
        elite_states = np.array([], dtype=float)
        elite_actions = np.array([], dtype=int)
        
        elite_indices = []
        
        for i in range(len(rewards_batch)):
            if rewards_batch[i] > reward_threshold:
                elite_indices.append(i)
        
        elite_indices = np.asarray(elite_indices)
        #print('indices', elite_indices)
        
        obs_num = np.shape(env.observation_space)[0]
        #print('obs num:', obs_num)
        
        for i in range(batch_size):
            for j in range(len(states_batch[i])):
                    
                if j == 0:
                    elite_states = np.append(elite_states, states_batch[i][0][0]) 
                    elite_actions = np.append(elite_actions, actions_batch[0][0])

                else:
                    elite_states = np.append(elite_states, states_batch[i][j])
                    elite_actions = np.append(elite_actions, actions_batch[i][j])

        return elite_states, elite_actions

In [10]:
n_states = np.shape(env.observation_space)[0]
n_actions = env.action_space.n

#print(n_states, n_actions)

#neural network
net = Net(n_states, hidden_size, n_actions)

In [11]:
# Check the initialized network

s = np.asarray([-100, -200, -410, -50])

s = s/TW*2

s_v = torch.FloatTensor([s]).reshape(((1,4)))

next_step = net.forward(s_v)

print(np.round((next_step[0]).detach().numpy(),decimals=2))
print(np.argmax((next_step[0]).detach().numpy()))

[0.12 0.12 0.09 0.12 0.1  0.11 0.08 0.15 0.12]
7


In [ ]:
#loss function
objective = nn.CrossEntropyLoss()

#optimisation function
optimizer = optim.Adam(params=net.parameters(), lr=learning_rate)

stats1 = np.array([])
stats2 = np.array([])
stats3 = np.array([])

for i in range(session_size):
    #generate new sessions
    batch_states,batch_actions,batch_rewards = net.generate_batch(env, batch_size, t_max)
    
    #print('batch_states', np.asarray(batch_states).reshape(-1,4))

    elite_states, elite_actions = net.filter_batch(batch_states,batch_actions,batch_rewards,percentile)
    
    #print('el_states', elite_states)
    elite_states = elite_states.reshape(-1,4)
    
    #print('el_states: reshaped', elite_states)
    #print('el_actions', elite_actions)
    
    optimizer.zero_grad()
    
    tensor_states = torch.FloatTensor(elite_states)
    tensor_actions = torch.LongTensor(elite_actions)

    action_scores_v = net(tensor_states)
    
    #print(action_scores_v)
    
    loss_v = objective(action_scores_v, tensor_actions)
    loss_v.backward()
    optimizer.step()

    #show results
    mean_reward, threshold = np.mean(batch_rewards), np.percentile(batch_rewards, percentile)
    print("%d: loss=%.5f, reward_mean=%.5f, reward_threshold=%.5f"% (i, loss_v.item(), mean_reward, threshold))
    
    stats1 = np.append(stats1, loss_v.item())
    stats2 = np.append(stats2, mean_reward)
    stats3 = np.append(stats3, threshold)
    
    #check if 
    if np.mean(batch_rewards)> completion_score:
        print("Environment has been successfully completed!")

Done: Array length error
Here is the problem! -> not every angle value is unique
r_array_mod [ 0.00000000e+00  0.00000000e+00 -8.65223728e-06 -1.30459610e-05
 -1.74866805e-05 -2.19759332e-05 -2.65153509e-05 -3.11066504e-05
 -3.57516549e-05 -4.04522934e-05 -4.52106093e-05 -5.00287883e-05
 -5.49091414e-05 -5.98541417e-05 -6.48664284e-05 -6.99488261e-05
 -7.51043640e-05 -8.03362889e-05 -8.56481037e-05 -9.10435802e-05
 -9.65268065e-05 -1.02102204e-04 -1.07774586e-04 -1.13549197e-04
 -1.19431765e-04 -1.25428593e-04 -1.31546600e-04 -1.37793448e-04
 -1.44177637e-04 -1.50708646e-04 -1.57397092e-04 -1.64254942e-04
 -1.71295764e-04 -1.78535027e-04 -1.85990534e-04 -1.93682892e-04
 -2.01636229e-04 -2.09879055e-04 -2.18445489e-04 -2.27376937e-04
 -2.36724455e-04 -2.46552256e-04 -2.56942979e-04 -2.68006112e-04
 -2.79892011e-04 -2.92817141e-04 -3.07113755e-04 -3.23341121e-04
 -3.42589761e-04 -3.67675027e-04]
0: loss=2.19518, reward_mean=17.72094, reward_threshold=67.14365
1: loss=2.19379, reward_mean

C:\Users\asus_strix_scar\Anaconda3\lib\site-packages\gym\envs\steering-optimizer\steering_optimizer\envs\github\steering-optimizer\steering_optimizer\envs\optimizer_env.py:32: RuntimeWarning: invalid value encountered in double_scalars
  chop = (angle*180/np.pi) % (np.sign(angle)*180)*np.pi/180


41: loss=1.52308, reward_mean=-36.50910, reward_threshold=-10.00000
42: loss=1.49094, reward_mean=-31.33960, reward_threshold=-10.00000
43: loss=1.46108, reward_mean=-29.92300, reward_threshold=-10.00000
44: loss=1.44522, reward_mean=-32.61300, reward_threshold=-10.00000
45: loss=1.42686, reward_mean=-33.33700, reward_threshold=-10.00000
46: loss=1.41017, reward_mean=-32.07460, reward_threshold=-10.00000
47: loss=1.40061, reward_mean=-38.53690, reward_threshold=-10.00000
48: loss=1.39581, reward_mean=-35.14021, reward_threshold=-10.00000
49: loss=1.38931, reward_mean=-34.78970, reward_threshold=-10.00000
50: loss=1.38519, reward_mean=-36.25770, reward_threshold=-10.00000
51: loss=1.38469, reward_mean=-34.65600, reward_threshold=-10.00000
52: loss=1.38071, reward_mean=-28.72340, reward_threshold=-10.00000
53: loss=1.37844, reward_mean=-32.56610, reward_threshold=-10.00000
54: loss=1.37736, reward_mean=-36.01800, reward_threshold=-10.00000
55: loss=1.37715, reward_mean=-31.80190, reward_

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

plt.figure()
plt.plot(stats1)
plt.figure()
plt.plot(stats2)
plt.figure()
plt.plot(stats3)

In [ ]:
s = np.asarray([-100, -200, -410, -50])

s = s/TW*2

s_v = torch.FloatTensor([s]).reshape(((1,4)))

next_step = net.forward(s_v)

print(np.round((next_step[0]).detach().numpy(),decimals=2))
print(np.round((next_step[0]).detach().numpy(),decimals=0))
print(np.argmax((next_step[0]).detach().numpy()))

In [ ]:
import gym.wrappers
env = gym.wrappers.Monitor(gym.make("steering_optimizer-v0"), directory="videos", force=True)
net.generate_batch(env, 1, t_max=5000)
env.close()

In [ ]:
print(net.parameters)